In [1]:
import numpy as n
from scipy.optimize import root
import Properties as prop
import pandas as pd

calctolerance=10**-7
calcmethod = "hybr"
KPD=0.99

#Таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#Таблица частичной нагрузки (входные данные)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

class vapor:
    def __init__(self, stream11, stream12,stream21,stream22):
        self.Gas_composition={'N2':100*gas_streams0.at[stream11,'N2'],'O2':100*gas_streams0.at[stream11,'O2'],'CO2':100*gas_streams0.at[stream11,'CO2'],'H2O':100*gas_streams0.at[stream11,'H2O'],'Ar':100*gas_streams0.at[stream11,'Ar']}
        self.H011 = gas_streams0.at[stream11,'H']
        self.H012 = gas_streams0.at[stream12,'H']
        self.H021 = water_streams0.at[stream21,'H']
        self.H022 = water_streams0.at[stream22,'H']
        self.G01  = gas_streams0.at[stream11,'G']
        self.G02  = water_streams0.at[stream21,'G']
        self.P01  = gas_streams0.at[stream11,'P']
        self.P02  = water_streams0.at[stream21,'P']
        self.H11  = gas_streams.at[stream11,'H']
        self.H21  = water_streams.at[stream21,'H']
        self.G1   = gas_streams.at[stream11,'G']
        self.P1   = gas_streams.at[stream11,'P']
        self.P2   = water_streams.at[stream21,'P']
    def calc(self):
        Q0     = self.G01*(self.H011-self.H012)*KPD
        T011   = prop.gas_T_HP(self.H011,self.P01,self.Gas_composition)
        T012   = prop.gas_T_HP(self.H012,self.P01,self.Gas_composition)
        T021   = prop.water_T_HP(self.H021,self.P02)
        T022   = prop.water_T_HP(self.H022,self.P02)
        T0np = prop.water_TSS_P(self.P2)
        dTmin0 = min(T011-T0np,T012-T0np)
        dTmax0 = max(T011-T0np,T012-T0np)
        LMTD0  = (dTmax0 - dTmin0) / (n.log(dTmax0/dTmin0))
        T01av  = (T011+T012)/2
        lambda01av = prop.gas_L_PT(self.P01,T01av,self.Gas_composition)
        Pr01av = prop.gas_Prandtl_PT(self.P01,T01av,self.Gas_composition)
        nu01av = prop.gas_KINVIS_PT(self.P01,T01av,self.Gas_composition)
        ro01av = prop.gas_D_PT(self.P01,T01av,self.Gas_composition)
        def T12sved(T12):
            T11 = prop.gas_T_HP(self.H11,self.P1,self.Gas_composition)
            T21 = prop.water_T_HP(self.H21,self.P2)
            H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
            Q = self.G1*(self.H11-H12)*KPD
            H22 = prop.water_HSS_P(self.P2)
            T22 = prop.water_T_HP(H22,self.P2)
            G2 = Q/(H22-self.H21)
            dTmin= min(T11-T22,T12-T21)
            dTmax= max(T11-T22,T12-T21)
            if dTmin<0 or dTmax<0 or dTmin==dTmax:   ##от зацикливания
                LMTD = (dTmax+dTmin)/2
            else:
                LMTD = (dTmax - dTmin) / (n.log(dTmax/dTmin))            
            q = Q0/Q
            dt = LMTD0/LMTD
            k = q/dt
            T1av = (T11+T12)/2
            lambda1av = prop.gas_L_PT(self.P1,T1av,self.Gas_composition)
            Pr1av = prop.gas_Prandtl_PT(self.P1,T1av,self.Gas_composition)
            nu1av = prop.gas_KINVIS_PT(self.P1,T1av,self.Gas_composition)
            ro1av = prop.gas_D_PT(self.P1,T1av,self.Gas_composition)
            kk = (lambda01av/lambda1av)*((Pr01av/Pr1av)**0.33)*(((self.G01/self.G1)*(ro1av/ro01av)*(nu1av/nu01av))**0.685)
            Qq = Q0 / (kk*dt)
            return ((Q-Qq)/Q*100)
        sol = root(T12sved, T012, method=calcmethod, tol=calctolerance)
        T12=float(sol.x)
        H12 = prop.gas_H_PT(self.P1,T12,self.Gas_composition)
        Q = self.G1*(self.H11-H12)*KPD
        H22 = prop.water_HSS_P(self.P2)
        T22 = prop.water_T_HP(H22,self.P2)
        G2 = Q/(H22-self.H21)
        gas_streams.loc[stream12,'T':'G'] = [T12,self.P1,H12,self.G1]
        gas_streams.loc[stream12,'N2':'Ar'] = [gas_streams0.at[stream11,'N2'],gas_streams0.at[stream11,'O2'],gas_streams0.at[stream11,'CO2'],gas_streams0.at[stream11,'H2O'],gas_streams0.at[stream11,'Ar']]
        water_streams.loc[stream22,'T':'G'] = [T22,self.P2,H22,G2]
        return [T12,self.P1,H12,self.G1,T22,self.P2,H22,G2]
            
stream11 = 'PEVD-IVD'
stream12 = 'IVD-EVD'
stream21 = 'EVD-IVD'
stream22 = 'IVD-PEVD'
IVD = vapor(stream11, stream12,stream21,stream22)

IVD.calc()



                    T      P            H          G
PEVD-DROS  496.800000  8.025  3391.256252  61.510000
IVD-PEVD   272.450253  5.717  2787.569821  41.509523
EVD-IVD    270.100000  5.717  1185.707737  41.420000


,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,921.649205,516.2,0.7803,0.1237,0.0401,0.0494,0.0065
PEVD-IVD,465.300000,0.1,856.463256,317.6,0.7803,0.1237,0.0401,0.0494,0.0065
IVD-EVD,275.141997,0.1,647.103902,317.6,0.7803,0.1237,0.0401,0.0494,0.0065
